# GROUP89 COMP5046 Assignment2

The code used in assignment2 is quoted from lectures or labs

# Data and input

## Data Download and Load

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#get the data from the drive
#test_without_labels data
id = '1bK08xtaH12-uQxsi6HmUdgXaLFnWxSiM'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_without_labels.csv')

#train data
id = '1ESroPT67jePx77ksvY3Y99UGj6FJ38VT'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')

#val data
id = '1bvrtMRCjgg1QppD_0Puq2OhBZ9_uZ5Wp'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('val.csv')


In [ ]:
import pandas as pd

# import train.csv, val.csv and test.cav
data_train_df = pd.read_csv('./train.csv')

data_test_df = pd.read_csv('./test_without_labels.csv')

data_val_df = pd.read_csv('./val.csv')



In [ ]:
# convert to lsit for sentence
data_train_sents = data_train_df['sents'].tolist()
data_val_sents = data_val_df['sents'].tolist()
data_test_sents = data_test_df['sents'].tolist()

In [ ]:
# split to list
data_train_sents_split = [x.split() for x in data_train_sents]
data_val_sents_split = [x.split() for x in data_val_sents]
data_test_sents_split = [x.split() for x in data_test_sents]

In [ ]:
# convert to lsit and split for labels
data_train_labels_split = [x.split() for x in data_train_df['labels'].tolist()]
data_val_labels_split = [x.split() for x in data_val_df['labels'].tolist()]

In [ ]:
print(data_train_df.shape)
print(data_val_df.shape)
print(data_test_df.shape)

(26078, 2)
(8705, 2)
(500, 1)


## Input Embedding

###  Syntactic Textual Feature Embedding:(Pos Tag)

In [ ]:
# pos-tag for train set
import spacy
from spacy.symbols import ORTH
nlp = spacy.load("en_core_web_sm")

def pos_dep_way(input):
  for i in input:
    sent = i.split()
    for j in sent:
      special = [{ORTH: j}]
      nlp.tokenizer.add_special_case(j,special)

  data_postag = []
  data_dep = []
  for sent in input:
    postag = []
    dep = []
    doc = nlp(sent)
    for token in doc:
      postag.append(token.tag_)
      dep.append(token.dep_)
    data_postag.append(postag)
    data_dep.append(dep)
  return data_postag,data_dep

data_train_postag,data_train_dep = pos_dep_way(data_train_sents)
data_val_postag,data_val_dep = pos_dep_way(data_val_sents)
data_test_postag,data_test_dep = pos_dep_way(data_test_sents)


In [ ]:
from gensim.models import Word2Vec
# Preprocess data for postag
postag_data = data_train_postag+data_val_postag+data_test_postag

# Build training model for postag
postag_model = Word2Vec(sentences=postag_data, size=100, window=3, min_count=0, workers=2, sg=1)

# Preprocess data for dependency
dep_data = data_train_dep+data_val_dep+data_test_dep

# Build training model for dependency
dep_model = Word2Vec(sentences=dep_data, size=100, window=3, min_count=0, workers=2, sg=1)

In [ ]:
data_train_postag[5]

['PRP', 'NN', 'XX', 'VBP', 'XX', 'IN', 'CD', 'NN']

### Semantic Textual Feature Embedding: (Word2Vec) 

In [ ]:
from gensim.models import Word2Vec
# Preprocess data for sentence
sents_data = data_train_sents_split+data_val_sents_split+data_test_sents_split

# Build training model for sentence
sents_model = Word2Vec(sentences=sents_data, size=100, window=3, min_count=0, workers=2, sg=1)

[['wow'], ['WTF'], ['wpe', 'wpe'], ['hahaha'], ['wtf'], ['i', 'cant', '[SEPA]', 'play', '[SEPA]', 'with', '4', 'trash'], ['bg'], ['#ERROR!'], ['gg', '[SEPA]', 'report', 'my', 'team', 'rat', '[SEPA]', 'please'], ['ez', 'mid'], ['hahah'], ['my', 'arrows', '[SEPA]', 'always', 'decent', '[SEPA]', 'fuck', 'u'], ['gh'], ['always', 'engage', 'at', 'bot', 'lc', 'cant', 'takle', 'bot', 'then', 'top'], ['cmon'], ['the', 'comeback', 'is', 'real', ':)'], ['him', 'mid', 'vs', 'me'], ['just', 'end', '[SEPA]', 'i', 'wan', 'nex', 'game'], ['g'], ['he', 'is', 'not', 'losing'], ['Pls', 'report', 'sb', 'thanks'], ['omg'], ['ggwp'], ['WP', 'cap'], ['lo', '[SEPA]', 'lol'], ['fuyckjerfe'], ['gg', 'noob', 'mid', 'invoker'], ['i', 'mean', 'not', 'everyone', 'on', 'ur', 'team', 'is', 'dumb', 'enough', 'to', 'rot', 'down', 'to', '2', 'hp'], ['8'], ['it', 'just', 'random'], [':/'], ['u', 'dead', '[SEPA]', 'i'], ['he', 'just', 'keeps', 'charhing', '[SEPA]', 'since', 'spectre', 'comited', 'ult', '[SEPA]', 'so', 'd

### Domain Feature Embedding: (3-letter suffix )

In [ ]:
# Preprocess data for sentence

letter_3 = []
for i in sents_data:
  data_input = [x[-3:] if len(x)>=3 else "-" for x in i]
  letter_3.append(data_input)

# Build training model for letter_3
letter_model = Word2Vec(sentences=letter_3, size=100, window=3, min_count=0, workers=2, sg=1)

###  Input Concatenation

In [ ]:
# Generate word_to_ix and tag_to_ix

word_to_ix = {}
for sentence in data_train_sents_split+data_val_sents_split+data_test_sents_split:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())


START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in data_train_labels_split+data_val_labels_split:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)


In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size+dep_model.vector_size+sents_model.vector_size+letter_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((sents_model[word],word_postag[word],word_dep[word],word_letter[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
embedding_matrix.shape

(14015, 400)

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)


# Slot Filling/Tagging Model

## Baseline model

In [ ]:
# Generate word_to_ix and tag_to_ix

word_to_ix = {}
for sentence in data_train_sents_split+data_val_sents_split+data_test_sents_split:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in data_train_labels_split+data_val_labels_split:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)


In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

word_emb_model = api.load("glove-twitter-25") 

EMBEDDING_DIM = 25

embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

[==================================================] 100.0% 104.8/104.8MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':


(14015, 25)

In [ ]:
# convert dataset into idxs

def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)


In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1

In [ ]:
# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 50

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 39155.50, train f1: 0.9501, val loss: 5987.54, val f1: 0.9473, time: 342.40s
Epoch:2, Training loss: 11792.82, train f1: 0.9822, val loss: 3131.63, val f1: 0.9764, time: 339.96s


## Stacked Seq2Seq model(Bi-LSTM)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers,stack):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.stack = stack

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)   
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)  
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)    
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats



    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)

Epoch:1, Training loss: 35796.65, train f1: 0.9829, val loss: 2745.40, val f1: 0.9796, time: 565.11s
Epoch:2, Training loss: 4977.68, train f1: 0.9916, val loss: 1959.35, val f1: 0.9862, time: 525.05s


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i).to(device))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1

In [ ]:
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,stack=0).to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 39730.05, train f1: 0.9794, val loss: 3114.63, val f1: 0.9766, time: 761.18s


KeyboardInterrupt: ignored

## Attention

In [ ]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cos_similar(self, query, key):
        sim_matrix = torch.bmm(query, key)
        a = torch.norm(query, p=2, dim=-1)
        b = torch.norm(key, p=2, dim=-2)
        sim_matrix /= a.unsqueeze(-1)
        sim_matrix /= b.unsqueeze(-2)
        return sim_matrix

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "Content-base":
            attention_weight = F.softmax(self.cos_similar(query, key), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        return lstm_feats, torch.argmax(lstm_feats,-1)

Epoch:1, Training loss: 11268.06, train f1: 0.9981, val loss: 1366.13, val f1: 0.9858, time: 568.82s
Epoch:2, Training loss: 230.77, train f1: 0.9997, val loss: 2150.09, val f1: 0.9848, time: 567.85s


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i).to(device))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1

In [ ]:
# Initialize Model

HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Content-base",stack=0).to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 15307.14, train f1: 0.9944, val loss: 1434.97, val f1: 0.9902, time: 1075.96s
Epoch:2, Training loss: 1498.13, train f1: 0.9982, val loss: 2491.24, val f1: 0.9921, time: 1074.37s


## CRF Attachment

In [ ]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cos_similar(self, query, key):
        sim_matrix = torch.bmm(query, key)
        a = torch.norm(query, p=2, dim=-1)
        b = torch.norm(key, p=2, dim=-2)
        sim_matrix /= a.unsqueeze(-1)
        sim_matrix /= b.unsqueeze(-2)
        return sim_matrix

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "Content-base":
            attention_weight = F.softmax(self.cos_similar(query, key), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

Epoch:1, Training loss: 11268.06, train f1: 0.9990, val loss: 1366.13, val f1: 0.9911, time: 668.35s
Epoch:2, Training loss: 230.77, train f1: 0.9997, val loss: 2150.09, val f1: 0.9879, time: 646.86s


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i).to(device))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1

In [ ]:
# Initialize Model

HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Content-base",stack=0).to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f,val f1T: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc,val_T, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

TypeError: ignored

# Evaluation Setup

## Embedding


1. only syntactic
2. only semantic
3. only 3-letter-suffix
4. combination of 1,2,3



### 1 only syntactic-PoS

In [ ]:
# 1
import spacy
from spacy.symbols import ORTH
nlp = spacy.load("en_core_web_sm")

def pos_dep_way(input):
  for i in input:
    sent = i.split()
    for j in sent:
      special = [{ORTH: j}]
      nlp.tokenizer.add_special_case(j,special)

  data_postag = []
  data_dep = []
  for sent in input:
    postag = []
    dep = []
    doc = nlp(sent)
    for token in doc:
      postag.append(token.tag_)
      dep.append(token.dep_)
    data_postag.append(postag)
    data_dep.append(dep)
  return data_postag,data_dep

data_train_postag,data_train_dep = pos_dep_way(data_train_sents)
data_val_postag,data_val_dep = pos_dep_way(data_val_sents)
data_test_postag,data_test_dep = pos_dep_way(data_test_sents)

In [ ]:
from gensim.models import Word2Vec
# Preprocess data for postag
postag_data = data_train_postag+data_val_postag+data_test_postag

# Build training model for postag
postag_model = Word2Vec(sentences=postag_data, size=100, window=3, min_count=0, workers=2, sg=1)

# Preprocess data for dependency
dep_data = data_train_dep+data_val_dep+data_test_dep

# Build training model for dependency
dep_model = Word2Vec(sentences=dep_data, size=100, window=3, min_count=0, workers=2, sg=1)

In [ ]:
from gensim.models import Word2Vec
# Preprocess data for sentence
sents_data = data_train_sents_split+data_val_sents_split+data_test_sents_split

# Build training model for sentence
sents_model = Word2Vec(sentences=sents_data, size=100, window=3, min_count=0, workers=2, sg=1)

In [ ]:
letter_3 = []
for i in sents_data:
  data_input = [x[-3:] if len(x)>=3 else "-" for x in i]
  letter_3.append(data_input)

# Build training model for letter_3
letter_model = Word2Vec(sentences=letter_3, size=100, window=3, min_count=0, workers=2, sg=1)

In [ ]:
word_to_ix = {}
for sentence in data_train_sents_split+data_val_sents_split+data_test_sents_split:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())


START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in data_train_labels_split+data_val_labels_split:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(word_postag[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
embedding_matrix.shape

(14015, 100)

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i).to(device))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1

In [ ]:
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 34050.45, train f1: 0.9847, val loss: 2762.80, val f1: 0.9817, time: 467.46s
Epoch:2, Training loss: 3638.58, train f1: 0.9975, val loss: 1996.50, val f1: 0.9918, time: 467.70s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item

In [ ]:
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
    

In [ ]:
real_f1(predicted, ground_truth)

C:0.9573430834856794,D:0.864321608040201,O:0.9999473268369765,P:0.9954268292682927,S:0.9819385912101144,SEPA:1.0,T:0.9509870660313138,noO:0.9809311712714872,total:0.991755111830665


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))


# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

              precision    recall  f1-score   support

           C     0.9975    0.9573    0.9770      1641
           D     0.9971    0.8618    0.9245       398
           O     0.9875    1.0000    0.9937     18985
           P     0.9995    0.9959    0.9977      3936
           S     0.9969    0.9816    0.9892      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9894    0.9523    0.9705      1469

    accuracy                         0.9918     33354
   macro avg     0.9954    0.9642    0.9790     33354
weighted avg     0.9919    0.9918    0.9918     33354



### 2 only syntactic - Dep

In [ ]:
EMBEDDING_DIM = dep_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(word_dep[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
embedding_matrix.shape

(14015, 100)

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)

In [ ]:
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())

In [ ]:

#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 35509.24, train f1: 0.9815, val loss: 2944.70, val f1: 0.9775, time: 471.44s
Epoch:2, Training loss: 3745.79, train f1: 0.9969, val loss: 2104.16, val f1: 0.9911, time: 471.77s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9597806215722121,D:0.7989949748743719,O:0.9999473268369765,P:0.9946646341463414,S:0.9801324503311258,SEPA:1.0,T:0.9530292716133424,noO:0.9789825318393764,total:0.9909156323079691


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9758    0.9580    0.9668      1641
           D     0.9691    0.7889    0.8698       398
           O     0.9875    0.9999    0.9937     18985
           P     0.9985    0.9949    0.9967      3936
           S     0.9982    0.9795    0.9888      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9936    0.9530    0.9729      1469

    accuracy                         0.9907     33354
   macro avg     0.9889    0.9535    0.9698     33354
weighted avg     0.9907    0.9907    0.9905     33354



### 3 only semantic - Word2Vec

In [ ]:
EMBEDDING_DIM = sents_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(sents_model[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
print(embedding_matrix.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(14015, 100)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 28827.11, train f1: 0.9877, val loss: 2299.61, val f1: 0.9846, time: 474.55s
Epoch:2, Training loss: 2823.20, train f1: 0.9983, val loss: 1880.07, val f1: 0.9921, time: 471.29s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9603900060938452,D:0.8919597989949749,O:1.0,P:0.9951727642276422,S:0.9798314268512944,SEPA:1.0,T:0.9557522123893806,noO:0.9819750852529752,total:0.9922348144150627


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9906    0.9598    0.9749      1641
           D     0.9860    0.8819    0.9310       398
           O     0.9878    1.0000    0.9939     18985
           P     0.9995    0.9952    0.9973      3936
           S     0.9994    0.9801    0.9897      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9950    0.9544    0.9743      1469

    accuracy                         0.9921     33354
   macro avg     0.9940    0.9673    0.9802     33354
weighted avg     0.9921    0.9921    0.9920     33354



### 4 only domain - 3-letter-suffix

In [ ]:
EMBEDDING_DIM = letter_model.vector_size
# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(word_letter[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
print(embedding_matrix.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(14015, 100)


In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 35005.29, train f1: 0.9878, val loss: 2493.32, val f1: 0.9840, time: 470.47s
Epoch:2, Training loss: 2987.61, train f1: 0.9985, val loss: 1888.86, val f1: 0.9921, time: 473.62s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9609993906154783,D:0.9045226130653267,O:0.9996839610218594,P:0.9956808943089431,S:0.9807344972907887,SEPA:1.0,T:0.953710006807352,noO:0.9825318393764354,total:0.9922947772381123


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9956    0.9592    0.9770      1641
           D     0.9809    0.9045    0.9412       398
           O     0.9877    0.9997    0.9937     18985
           P     0.9997    0.9957    0.9977      3936
           S     0.9994    0.9807    0.9900      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9986    0.9558    0.9767      1469

    accuracy                         0.9923     33354
   macro avg     0.9946    0.9708    0.9823     33354
weighted avg     0.9924    0.9923    0.9923     33354



### 5 3-letter + w2V

In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = sents_model.vector_size+letter_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((sents_model[word],word_letter[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
print(embedding_matrix.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(14015, 200)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 27389.53, train f1: 0.9898, val loss: 2185.57, val f1: 0.9861, time: 528.11s
Epoch:2, Training loss: 2763.66, train f1: 0.9978, val loss: 1866.67, val f1: 0.9918, time: 523.91s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9585618525289458,D:0.8517587939698492,O:0.9999473268369765,P:0.9956808943089431,S:0.981637567730283,SEPA:1.0,T:0.9516678012253234,noO:0.9807919827406222,total:0.9916951490076152


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9949    0.9592    0.9767      1641
           D     1.0000    0.8693    0.9301       398
           O     0.9873    1.0000    0.9936     18985
           P     0.9977    0.9957    0.9967      3936
           S     0.9994    0.9825    0.9909      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9950    0.9483    0.9711      1469

    accuracy                         0.9919     33354
   macro avg     0.9963    0.9650    0.9799     33354
weighted avg     0.9920    0.9919    0.9918     33354



### 6 3-letter + w2V + pos

In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size+sents_model.vector_size+letter_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((sents_model[word],word_postag[word],word_letter[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 24532.00, train f1: 0.9893, val loss: 2015.83, val f1: 0.9857, time: 481.74s
Epoch:2, Training loss: 2569.98, train f1: 0.9980, val loss: 1736.28, val f1: 0.9920, time: 484.16s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9609993906154783,D:0.8793969849246231,O:0.9999473268369765,P:0.9956808943089431,S:0.9801324503311258,SEPA:1.0,T:0.9530292716133424,noO:0.9816271139258125,total:0.9920549259459135


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9893    0.9610    0.9750      1641
           D     0.9888    0.8844    0.9337       398
           O     0.9877    1.0000    0.9938     18985
           P     0.9990    0.9962    0.9976      3936
           S     1.0000    0.9801    0.9900      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9979    0.9503    0.9735      1469

    accuracy                         0.9921     33354
   macro avg     0.9947    0.9674    0.9805     33354
weighted avg     0.9921    0.9921    0.9920     33354



### 7 All

In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size+dep_model.vector_size+sents_model.vector_size+letter_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((sents_model[word],word_postag[word],word_dep[word],word_letter[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

NameError: ignored

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 23362.85, train f1: 0.9920, val loss: 1965.19, val f1: 0.9880, time: 485.94s
Epoch:2, Training loss: 2478.75, train f1: 0.9986, val loss: 1573.48, val f1: 0.9927, time: 484.60s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9603900060938452,D:0.8994974874371859,O:1.0,P:0.9961890243902439,S:0.9825406381697772,SEPA:1.0,T:0.9543907420013614,noO:0.9829494049690306,total:0.9926545541764106


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9962    0.9592    0.9773      1641
           D     0.9972    0.9095    0.9514       398
           O     0.9879    0.9998    0.9938     18985
           P     0.9997    0.9962    0.9980      3936
           S     0.9985    0.9825    0.9904      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9993    0.9544    0.9763      1469

    accuracy                         0.9926     33354
   macro avg     0.9970    0.9717    0.9839     33354
weighted avg     0.9927    0.9926    0.9925     33354



### 8 PoS+Dep

In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size+dep_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((word_postag[word],word_dep[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 31007.14, train f1: 0.9860, val loss: 2511.84, val f1: 0.9826, time: 482.69s
Epoch:2, Training loss: 3328.30, train f1: 0.9980, val loss: 1798.67, val f1: 0.9923, time: 483.03s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9567336989640464,D:0.8894472361809045,O:0.9999473268369765,P:0.9956808943089431,S:0.9822396146899458,SEPA:1.0,T:0.9496255956432947,noO:0.98155751966038,total:0.9920249445343887


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9981    0.9573    0.9773      1641
           D     0.9699    0.8894    0.9279       398
           O     0.9876    1.0000    0.9937     18985
           P     0.9990    0.9957    0.9973      3936
           S     0.9991    0.9819    0.9904      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9964    0.9496    0.9725      1469

    accuracy                         0.9921     33354
   macro avg     0.9929    0.9677    0.9799     33354
weighted avg     0.9921    0.9921    0.9920     33354



## Stack Layer

### 0

In [ ]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size+dep_model.vector_size+sents_model.vector_size+letter_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((sents_model[word],word_postag[word],word_dep[word],word_letter[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i).to(device))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1
# Model


import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers,stack):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.stack = stack

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)   
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)  
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)    
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats



    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,stack=0).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 23217.94, train f1: 0.9904, val loss: 2101.69, val f1: 0.9866, time: 567.37s
Epoch:2, Training loss: 2565.59, train f1: 0.9986, val loss: 1610.46, val f1: 0.9924, time: 551.40s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9579524680073126,D:0.8969849246231156,O:0.9998419805109297,P:0.9964430894308943,S:0.981637567730283,SEPA:1.0,T:0.95643294758339,noO:0.9826710279073004,total:0.9924446842957366


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9943    0.9592    0.9764      1641
           D     0.9917    0.8970    0.9420       398
           O     0.9879    0.9998    0.9938     18985
           P     0.9987    0.9959    0.9973      3936
           S     0.9988    0.9816    0.9901      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     1.0000    0.9558    0.9774      1469

    accuracy                         0.9924     33354
   macro avg     0.9959    0.9699    0.9824     33354
weighted avg     0.9925    0.9924    0.9924     33354



### 1


In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers,stack):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.stack = stack

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)   
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)  
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)    
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats



    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,stack=1).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 35796.65, train f1: 0.9829, val loss: 2745.40, val f1: 0.9796, time: 565.11s
Epoch:2, Training loss: 4977.68, train f1: 0.9916, val loss: 1959.35, val f1: 0.9862, time: 525.05s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9512492382693479,D:0.7537688442211056,O:0.9954174348169608,P:0.9951727642276422,S:0.9765201685731487,SEPA:1.0,T:0.9407760381211708,noO:0.9748068759134247,total:0.9865383462253403


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9506    0.9500    0.9503      1641
           D     0.8997    0.7437    0.8143       398
           O     0.9869    0.9962    0.9915     18985
           P     0.9977    0.9959    0.9968      3936
           S     0.9923    0.9759    0.9841      3322
        SEPA     0.9992    1.0000    0.9996      3603
           T     0.9739    0.9408    0.9571      1469

    accuracy                         0.9868     33354
   macro avg     0.9715    0.9432    0.9562     33354
weighted avg     0.9866    0.9868    0.9866     33354



### 3


In [ ]:
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers,stack):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.stack = stack

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)   
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)  
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)    
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats



    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,stack=3).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

NameError: ignored

In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9037172455819622,D:0.0,O:0.9954701079799843,P:0.985010162601626,S:0.9382901866345575,SEPA:1.0,T:0.8400272294077604,noO:0.9265780499686825,total:0.965791209450141


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.8367    0.9086    0.8712      1641
           D     0.0000    0.0000    0.0000       398
           O     0.9787    0.9963    0.9874     18985
           P     0.9990    0.9858    0.9923      3936
           S     0.9946    0.9398    0.9664      3322
        SEPA     0.9950    1.0000    0.9975      3603
           T     0.7783    0.8482    0.8117      1469

    accuracy                         0.9671     33354
   macro avg     0.7975    0.8112    0.8038     33354
weighted avg     0.9569    0.9671    0.9618     33354



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Attention

### 1 Scale Dot Product

In [ ]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "General":
            energy = torch.nn.Linear(self.hidden_dim, self.hidden_dim).to(device)
            energy = energy(self.K(squ_lstm)).transpose(1,0).unsqueeze(0)
            attention_weight = F.softmax(torch.bmm(query, energy), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Scale Dot Product",stack=0).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 11440.41, train f1: 0.9975, val loss: 1336.29, val f1: 0.9883, time: 567.35s
Epoch:2, Training loss: 407.91, train f1: 0.9997, val loss: 1966.16, val f1: 0.9880, time: 595.86s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9609993906154783,D:0.9246231155778895,O:0.990782196470898,P:0.9969512195121951,S:0.981637567730283,SEPA:1.0,T:0.95643294758339,noO:0.9839237246850859,total:0.987827546920909


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9900    0.9610    0.9753      1641
           D     1.0000    0.9246    0.9608       398
           O     0.9890    0.9910    0.9900     18985
           P     0.9585    0.9972    0.9775      3936
           S     1.0000    0.9819    0.9909      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9965    0.9564    0.9760      1469

    accuracy                         0.9880     33354
   macro avg     0.9906    0.9732    0.9815     33354
weighted avg     0.9882    0.9880    0.9880     33354



### 2 Dot Product

In [ ]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "General":
            energy = torch.nn.Linear(self.hidden_dim, self.hidden_dim).to(device)
            energy = energy(self.K(squ_lstm)).transpose(1,0).unsqueeze(0)
            attention_weight = F.softmax(torch.bmm(query, energy), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Dot Product",stack=0).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 11538.50, train f1: 0.9971, val loss: 1440.82, val f1: 0.9849, time: 574.50s
Epoch:2, Training loss: 429.99, train f1: 0.9998, val loss: 1882.67, val f1: 0.9866, time: 548.03s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.967702620353443,D:0.9246231155778895,O:0.9889386357650777,P:0.9961890243902439,S:0.9825406381697772,SEPA:1.0,T:0.95643294758339,noO:0.9846892616048437,total:0.9871079930443125


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9215    0.9659    0.9432      1641
           D     0.9840    0.9246    0.9534       398
           O     0.9899    0.9871    0.9885     18985
           P     0.9689    0.9962    0.9823      3936
           S     1.0000    0.9825    0.9912      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9943    0.9564    0.9750      1469

    accuracy                         0.9860     33354
   macro avg     0.9798    0.9733    0.9762     33354
weighted avg     0.9863    0.9860    0.9860     33354



### 3 Content-base

In [ ]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cos_similar(self, query, key):
        sim_matrix = torch.bmm(query, key)
        a = torch.norm(query, p=2, dim=-1)
        b = torch.norm(key, p=2, dim=-2)
        sim_matrix /= a.unsqueeze(-1)
        sim_matrix /= b.unsqueeze(-2)
        return sim_matrix

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "Content-base":
            attention_weight = F.softmax(self.cos_similar(query, key), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Content-base",stack=0).to(device)
optimizer = optim.Adam(model.parameters())


#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.967702620353443,D:0.9246231155778895,O:0.9846721095601791,P:0.9964430894308943,S:0.9825406381697772,SEPA:1.0,T:0.9557522123893806,noO:0.9846892616048437,total:0.9846794987107993


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.8472    0.9665    0.9029      1641
           D     1.0000    0.9246    0.9608       398
           O     0.9899    0.9847    0.9873     18985
           P     0.9909    0.9964    0.9937      3936
           S     0.9997    0.9825    0.9910      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     1.0000    0.9558    0.9774      1469

    accuracy                         0.9846     33354
   macro avg     0.9754    0.9729    0.9733     33354
weighted avg     0.9857    0.9846    0.9849     33354



## CRF

In [ ]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cos_similar(self, query, key):
        sim_matrix = torch.bmm(query, key)
        a = torch.norm(query, p=2, dim=-1)
        b = torch.norm(key, p=2, dim=-2)
        sim_matrix /= a.unsqueeze(-1)
        sim_matrix /= b.unsqueeze(-2)
        return sim_matrix

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "Content-base":
            attention_weight = F.softmax(self.cos_similar(query, key), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Content-base",stack=0).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 11268.06, train f1: 0.9990, val loss: 1366.13, val f1: 0.9911, time: 668.35s
Epoch:2, Training loss: 230.77, train f1: 0.9997, val loss: 2150.09, val f1: 0.9879, time: 646.86s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.9664838513101767,D:0.9246231155778895,O:0.9899920990255465,P:0.9961890243902439,S:0.9825406381697772,SEPA:1.0,T:0.9557522123893806,noO:0.9844804788085462,total:0.9876176770402351


In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, yy = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))

              precision    recall  f1-score   support

           C     0.9046    0.9647    0.9336      1641
           D     1.0000    0.9246    0.9608       398
           O     0.9890    0.9904    0.9897     18985
           P     0.9919    0.9959    0.9939      3936
           S     0.9997    0.9825    0.9910      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     1.0000    0.9558    0.9774      1469

    accuracy                         0.9877     33354
   macro avg     0.9836    0.9734    0.9781     33354
weighted avg     0.9881    0.9877    0.9878     33354



# Final Best Model (embedding+attention)

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#get the data from the drive
#test_without_labels data
id = '1bK08xtaH12-uQxsi6HmUdgXaLFnWxSiM'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_without_labels.csv')

#train data
id = '1ESroPT67jePx77ksvY3Y99UGj6FJ38VT'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')

#val data
id = '1bvrtMRCjgg1QppD_0Puq2OhBZ9_uZ5Wp'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('val.csv')


In [2]:
import pandas as pd

# import train.csv, val.csv and test.cav
data_train_df = pd.read_csv('./train.csv')

data_test_df = pd.read_csv('./test_without_labels.csv')

data_val_df = pd.read_csv('./val.csv')



In [3]:
# convert to lsit for sentence
data_train_sents = data_train_df['sents'].tolist()
data_val_sents = data_val_df['sents'].tolist()
data_test_sents = data_test_df['sents'].tolist()

In [4]:
# split to list
data_train_sents_split = [x.split() for x in data_train_sents]
data_val_sents_split = [x.split() for x in data_val_sents]
data_test_sents_split = [x.split() for x in data_test_sents]

In [5]:
# convert to lsit and split for labels
data_train_labels_split = [x.split() for x in data_train_df['labels'].tolist()]
data_val_labels_split = [x.split() for x in data_val_df['labels'].tolist()]

In [6]:
print(data_train_df.shape)
print(data_val_df.shape)
print(data_test_df.shape)

(26078, 2)
(8705, 2)
(500, 1)


In [10]:
# pos-tag for train set
import spacy
from spacy.symbols import ORTH
nlp = spacy.load("en_core_web_sm")

def pos_dep_way(input):
  for i in input:
    sent = i.split()
    for j in sent:
      special = [{ORTH: j}]
      nlp.tokenizer.add_special_case(j,special)

  data_postag = []
  data_dep = []
  for sent in input:
    postag = []
    dep = []
    doc = nlp(sent)
    for token in doc:
      postag.append(token.tag_)
      dep.append(token.dep_)
    data_postag.append(postag)
    data_dep.append(dep)
  return data_postag,data_dep

data_train_postag,data_train_dep = pos_dep_way(data_train_sents)
data_val_postag,data_val_dep = pos_dep_way(data_val_sents)
data_test_postag,data_test_dep = pos_dep_way(data_test_sents)


In [11]:
from gensim.models import Word2Vec
# Preprocess data for postag
postag_data = data_train_postag+data_val_postag+data_test_postag

# Build training model for postag
postag_model = Word2Vec(sentences=postag_data, size=100, window=3, min_count=0, workers=2, sg=1)

# Preprocess data for dependency
dep_data = data_train_dep+data_val_dep+data_test_dep

# Build training model for dependency
dep_model = Word2Vec(sentences=dep_data, size=100, window=3, min_count=0, workers=2, sg=1)

In [12]:
from gensim.models import Word2Vec
# Preprocess data for sentence
sents_data = data_train_sents_split+data_val_sents_split+data_test_sents_split

# Build training model for sentence
sents_model = Word2Vec(sentences=sents_data, size=100, window=3, min_count=0, workers=2, sg=1)

In [13]:
# Preprocess data for sentence

letter_3 = []
for i in sents_data:
  data_input = [x[-3:] if len(x)>=3 else "-" for x in i]
  letter_3.append(data_input)

# Build training model for letter_3
letter_model = Word2Vec(sentences=letter_3, size=100, window=3, min_count=0, workers=2, sg=1)

In [14]:
# Generate word_to_ix and tag_to_ix

word_to_ix = {}
for sentence in data_train_sents_split+data_val_sents_split+data_test_sents_split:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())


START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in data_train_labels_split+data_val_labels_split:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)


In [15]:
# Generate Embedding Matrix
import gensim.downloader as api
import numpy as np

EMBEDDING_DIM = postag_model.vector_size+dep_model.vector_size+sents_model.vector_size+letter_model.vector_size

# Add the second pre-trained word embedding to the Embedding lookup table via concatenation
embedding_matrix = []

word_postag = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_postag[sents_data[i][j]] = postag_model[postag_data[i][j]]

word_dep = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_dep[sents_data[i][j]] = dep_model[dep_data[i][j]]

word_letter = {}
for i in range(len(sents_data)):
  for j in range(len(sents_data[i])) :
    word_letter[sents_data[i][j]] = letter_model[letter_3[i][j]]

for word in word_list:
    try:
        embedding_matrix.append(np.concatenate((sents_model[word],word_postag[word],word_dep[word],word_letter[word]),0))
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [16]:
embedding_matrix.shape

(14015, 400)

In [17]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(data_train_sents_split,word_to_ix)
train_output_index = to_index(data_train_labels_split,tag_to_ix)
val_input_index = to_index(data_val_sents_split,word_to_ix)
val_output_index = to_index(data_val_labels_split,tag_to_ix)
test_input_index = to_index(data_test_sents_split,word_to_ix)
import numpy as np
from sklearn.metrics import f1_score
def cal_acc(model, input_index, output_index):
    predicted = [int(x) for i in input_index for x in model(torch.tensor(i).to(device))[1]]
    ground_truth = []
    for item in output_index:
      ground_truth = ground_truth + item
    

    f1 = f1_score(ground_truth,predicted, average='micro')
    return predicted, ground_truth, f1

In [18]:
from operator import concat
# Model
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim,layers, method, stack):
        super(BiLSTM_attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.layers = layers
        self.method = method
        self.stack = stack
        self.Q = nn.Linear(hidden_dim, hidden_dim)
        self.K = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, hidden_dim)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)
        
        self.stacklstm = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cos_similar(self, query, key):
        sim_matrix = torch.bmm(query, key)
        a = torch.norm(query, p=2, dim=-1)
        b = torch.norm(key, p=2, dim=-2)
        sim_matrix /= a.unsqueeze(-1)
        sim_matrix /= b.unsqueeze(-2)
        return sim_matrix

    def _get_lstm_features(self, sentence, method):
        self.hidden = self.init_hidden()

        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        for i in range(self.stack):
            self.hidden = self.init_hidden()
            lstm_out, self.hidden = self.stacklstm(lstm_out, self.hidden)

        # add attention to bilstm
    
        squ_lstm = lstm_out.squeeze(1)
        query = self.Q(squ_lstm).unsqueeze(0)
        key = self.K(squ_lstm).transpose(1,0).unsqueeze(0)
        value = self.V(squ_lstm).unsqueeze(0)
        if method == "Scale Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key)/torch.sqrt(torch.Tensor([self.hidden_dim]).to(device)), dim=-1)
        elif method == "Dot Product":
            attention_weight = F.softmax(torch.bmm(query, key), dim=-1)
        elif method == "Content-base":
            attention_weight = F.softmax(self.cos_similar(query, key), dim=-1)

        lstm_out = torch.cat((torch.bmm(attention_weight, value).transpose(1,0), lstm_out), -1)
        lstm_out = lstm_out.view(len(sentence), 2*self.hidden_dim)
        
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence,self.method)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        # Get the emission scores from the BiLSTM

        lstm_feats = self._get_lstm_features(sentence,self.method)

        return lstm_feats, torch.argmax(lstm_feats,-1)
# Initialize Model
HIDDEN_DIM = 50
layers = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,layers,method="Content-base",stack=0).to(device)
optimizer = optim.Adam(model.parameters())
#Each epoch will take about 10-20 minutes

import datetime
from sklearn.metrics import f1_score
for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train f1: %.4f, val loss: %.2f, val f1: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section
# Please make sure you keep your own running log for submission

Epoch:1, Training loss: 11468.76, train f1: 0.9979, val loss: 1367.68, val f1: 0.9871, time: 610.84s
Epoch:2, Training loss: 257.98, train f1: 0.9998, val loss: 1888.62, val f1: 0.9908, time: 606.31s


In [ ]:
predicted = [int(x) for i in val_input_index for x in model(torch.tensor(i).to(device))[1]]
ground_truth = []
for item in val_output_index:
    ground_truth = ground_truth + item
predicted = np.array(predicted)
ground_truth = np.array(ground_truth)

def real_f1(predicted, ground_truth):
    f1_C=f1_score(predicted[ground_truth==tag_to_ix['C']],ground_truth[ground_truth==tag_to_ix['C']],average = 'micro')
    f1_D=f1_score(predicted[ground_truth==tag_to_ix['D']],ground_truth[ground_truth==tag_to_ix['D']],average = 'micro')
    f1_O=f1_score(predicted[ground_truth==tag_to_ix['O']],ground_truth[ground_truth==tag_to_ix['O']],average = 'micro')
    f1_P=f1_score(predicted[ground_truth==tag_to_ix['P']],ground_truth[ground_truth==tag_to_ix['P']],average = 'micro')
    f1_S=f1_score(predicted[ground_truth==tag_to_ix['S']],ground_truth[ground_truth==tag_to_ix['S']],average = 'micro')
    f1_Sepa=f1_score(predicted[ground_truth==tag_to_ix['SEPA']],ground_truth[ground_truth==tag_to_ix['SEPA']],average = 'micro')
    f1_T=f1_score(predicted[ground_truth==tag_to_ix['T']],ground_truth[ground_truth==tag_to_ix['T']],average = 'micro')
    f1_noO=f1_score(predicted[ground_truth!=tag_to_ix['O']],ground_truth[ground_truth!=tag_to_ix['O']],average = 'micro')
    f1_total=f1_score(predicted,ground_truth,average = 'micro')
    print("C:{},D:{},O:{},P:{},S:{},SEPA:{},T:{},noO:{},total:{}".format(f1_C,f1_D,f1_O,f1_P,f1_S,f1_Sepa,f1_T,f1_noO,f1_total))
real_f1(predicted, ground_truth)

C:0.967702620353443,D:0.9246231155778895,O:0.9846721095601791,P:0.9964430894308943,S:0.9825406381697772,SEPA:1.0,T:0.9557522123893806,noO:0.9846892616048437,total:0.9846794987107993


# Prediction

In [ ]:
# Prediction for testset label
predicted = [int(x) for i in test_input_index for x in model(torch.tensor(i).to(device))[1]]

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]
y_decode = decode_output(predicted)

In [ ]:
# save in the .csv file
output_df = pd.DataFrame(y_decode, columns = ['Predicted'])
output_df.to_csv('./test_output.csv', sep=",", float_format='%d',index_label="Id")